# Модель предсказания UpLift для компании "Лента"

In [1]:
! pip install CatBoost
! pip install LightGBM

     |████████████████████████████████| 69.2 MB 77 kB/s  eta 0:00:01
     |████████████████████████████████| 13.2 MB 24.2 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=265f2205483d21643f6f6527c662ebbe210aeb9b53baa32f04ea1e70774d8878
  Stored in directory: /home/lantana/.cache/pip/wheels/ac/cb/8a/b27bf6323e2f4c462dcbf77d70b7c5e7868a7fbe12871770cf
Successfully built retrying


In [21]:
! pip install -U pandasql

Processing /home/lantana/.cache/pip/wheels/a6/64/11/62d5f7b88421a5d80068ac9937fe2915e497da3eba900c8f59/pandasql-0.7.3-py3-none-any.whl
  Using cached SQLAlchemy-1.4.17-cp36-cp36m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.5 MB)


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
#import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy import stats as st
import matplotlib.pyplot as plt
#import seaborn as sns
pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(precision=3,suppress=True)
import warnings
warnings.simplefilter("ignore")
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None
import time
#from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
import datetime

In [22]:
import pandasql as ps

## Подготовка данных

In [3]:
checks = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210518_checks.csv')

In [4]:
hierarchy = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210518_hierarchy.csv')

In [5]:
uplift = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210518_uplift.csv')

In [6]:
offers = pd.read_csv('https://palitrastorage.blob.core.windows.net/data/20210521_offers.csv')

Создадим функцию для перевода данных из формата int в формат даты.

In [7]:
def datification(weird_date):
    new_date = datetime.datetime.strptime(weird_date, '%Y%m%d')
    return new_date

In [8]:
offers['start_date'] = offers['start_date'].astype(str)
offers['start_date'] = offers['start_date'].apply(datification)
offers['end_date'] = offers['end_date'].astype(str)
offers['end_date'] = offers['end_date'].apply(datification)

In [9]:
offers.head()

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train
1,Facades,F2,e87788b585ead792a4d40f7de18f2f76,2020-01-02,2020-01-08,train
2,Facades,F9,5ae92b5e465d630285677e80b43fe5f7,2020-01-16,2020-01-22,train
3,Facades,F45,0790b472112a765339f0e5c3f66816ee,2020-03-19,2020-03-25,train
4,Facades,F191,e87788b585ead792a4d40f7de18f2f76,2020-11-12,2020-11-18,test


In [10]:
checks['day'] = checks['day'].astype(str)
checks['day'] = checks['day'].apply(datification)

In [11]:
checks.head()

,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name
0,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b17c870027bb4a22e3aedb971bc00def,b3deb0286313f0b888c0eac49580cc23,1,5.000,133.400,249.450,152f1b77a32508570e2745daf9ce7aec
1,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b9ef00e971d0fb459a765708efba70c0,680a96609351230ed00e1d1090371cbc,2,1.000,96.770,118.990,152f1b77a32508570e2745daf9ce7aec
2,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b75147feae4cf8ec07fb5f2cef855def,7215ee9c7d9dc229d2921a40e899ec5f,5,2.000,2.340,7.580,152f1b77a32508570e2745daf9ce7aec
3,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,b25e0d42165d246d23cbf4d3efcdf583,7215ee9c7d9dc229d2921a40e899ec5f,4,1.000,23.590,39.990,152f1b77a32508570e2745daf9ce7aec
4,d9da50b77962ad6401d0527db5b65b04,2019-12-30,25bbdcd06c32d477f7fa1c3e4a91b032,c0b3f9c2215f924e27a6039617999c60,07:28:39,a537d2eef30d47f9b11d655421ca93fb,05c93d4fa9bdbe70b5d2204ce9548ca9,3,1.000,24.020,39.990,152f1b77a32508570e2745daf9ce7aec


In [12]:
checks['selling_price_per_unit'] = checks['selling_price'] / checks['num_sales']

In [13]:
checks['supplier_price_per_unit'] = checks['supplier_price'] / checks['num_sales']

In [14]:
checks['promo_id'].value_counts().sort_values(ascending = False)

7215ee9c7d9dc229d2921a40e899ec5f    10881653
c4c67f412855bab90c566139cb2b3d83      182037
47b5739208ae99fe23f06fedb3ee8ea7       74030
d9ca24d75e5cecc5a01c452cc8b8288e       72601
7071bf06286a55deac8fa9619ae3517a       71467
                                      ...   
58f111b459b99bea2a0e84bd1ec4c549           1
cd55dfef16ae5268abd00c52a0ffd764           1
2cd8dfec384621210d8015baca2983b6           1
861ffe94a812a5f4c257e33785ca16e3           1
e6b2b1d153dd703c63c79aef2b88c42b           1
Name: promo_id, Length: 176939, dtype: int64

Больше всего товаров продаётся без акции. Заменим это значение promo_id на no_offers

In [16]:
checks['promo_id'] = checks['promo_id'].mask(checks['promo_id'] == '7215ee9c7d9dc229d2921a40e899ec5f', 'no_offers')

In [17]:
checks['promo_id'].value_counts().sort_values(ascending = False)

no_offers                           10881653
c4c67f412855bab90c566139cb2b3d83      182037
47b5739208ae99fe23f06fedb3ee8ea7       74030
d9ca24d75e5cecc5a01c452cc8b8288e       72601
7071bf06286a55deac8fa9619ae3517a       71467
                                      ...   
5f4919dcc81c1aa75354284478eac6e7           1
c110a10c4f7bb7fb6203600f7c5123aa           1
0e7b7b15e87279537f85541f55ce7bfb           1
11220e6400a74115ab9874459bd2a1a8           1
e6b2b1d153dd703c63c79aef2b88c42b           1
Name: promo_id, Length: 176939, dtype: int64

Добавим к таблице checks данные по иерархии товаров.

In [18]:
checks_merged1 = checks.merge(hierarchy, how='outer', on = ['sku'])

In [19]:
offers['duration'] = offers['end_date'] - offers['start_date'] + pd.to_timedelta('1 day')

Для определения предакционного периода введём дату pre_start_datepre_start_date

In [20]:
offers['pre_start_date'] = offers['start_date'] - offers['duration']

Выполним sql-запрос для отбора значений в таблице checks_merged, входящих в перечень акционных товаров и попадающих во временной интервал проведения акции

In [ ]:
sqlcode = '''
select *
from checks
inner join offers on checks.sku=offers.sku
where checks.day >= offers.start_date and checks.day <= offers.end_date
'''

checks_merged = ps.sqldf(sqlcode,locals())

In [ ]:
checks_merged.to_csv('checks_merged.csv')

In [23]:
checks_merged = pd.read_csv('checks_merged.csv')

Для быстроты обработки следующего запроса разобьём данные в таблице с чеками на 2 части. И по каждой выполним sql-запрос для выделения значений, входящих в перечень акционных товаров и попадающих в предакционный временной интервал.

In [ ]:
checks_part1 = checks.head(10739630)

In [ ]:
checks_part2 = checks.tail(10739630)

In [ ]:
sqlcode = '''
select *
from checks_part1
inner join offers on checks_part1.sku=offers.sku
where checks_part1.day >= offers.pre_start_date and checks_part1.day < offers.start_date
'''

checks_non_action1 = ps.sqldf(sqlcode,locals())

In [ ]:
checks_non_action1.to_csv('checks_non_action1.csv')

In [25]:
checks_non_action1 = pd.read_csv('checks_non_action1.csv')

In [ ]:
sqlcode = '''
select *
from checks_part2
inner join offers on checks_part2.sku=offers.sku
where checks_part2.day >= offers.pre_start_date and checks_part2.day < offers.start_date
'''

checks_non_action2 = ps.sqldf(sqlcode,locals())

In [ ]:
checks_non_action2.to_csv('checks_non_action2.csv')

In [26]:
checks_non_action2 = pd.read_csv('checks_non_action2.csv')

Объединим данные в один предакционный датафрейм.

In [27]:
checks_non_action = pd.concat([checks_non_action1, checks_non_action2], axis = 0, sort = False)

In [28]:
checks_non_action

,Unnamed: 0,client_id,day,shop_id,check_id,time,sku,promo_id,check_pos,num_sales,supplier_price,selling_price,region_name,selling_price_per_unit,supplier_price_per_unit,Promo_type,Offer_ID,sku.1,start_date,end_date,train_test_group,duration,pre_start_date
0,0,2ef5f8b86d8ccc8fe43e60cb0c1a37e1,2019-12-30 00:00:00.000000,25bbdcd06c32d477f7fa1c3e4a91b032,ef0bbb1b6e5b36bdf8763f0f739fe9ba,14:29:59,689b1532a1ed565a9e9baeed3624b2b4,0,4,1.024,69.410,76.790,152f1b77a32508570e2745daf9ce7aec,74.990,67.783,Facades,F1,689b1532a1ed565a9e9baeed3624b2b4,2020-01-02 00:00:00.000000,2020-01-08 00:00:00.000000,train,604800000000000,2019-12-26 00:00:00.000000
1,1,2ef5f8b86d8ccc8fe43e60cb0c1a37e1,2019-12-30 00:00:00.000000,25bbdcd06c32d477f7fa1c3e4a91b032,ef0bbb1b6e5b36bdf8763f0f739fe9ba,14:29:59,689b1532a1ed565a9e9baeed3624b2b4,0,4,1.024,69.410,76.790,152f1b77a32508570e2745daf9ce7aec,74.990,67.783,Facades,F252,689b1532a1ed565a9e9baeed3624b2b4,2020-01-02 00:00:00.000000,2020-01-08 00:00:00.000000,train,604800000000000,2019-12-26 00:00:00.000000
2,2,58043d7e5ed135225aded3e9fabe6e8d,2019-12-30 00:00:00.000000,25bbdcd06c32d477f7fa1c3e4a91b032,68286dd828d92adb9299ca75e57a45ac,15:33:57,8230aaeb57e52b80413424a7ac1b5f88,25685af065808d88a7d8f16dbed8a87e,2,1.000,40.190,69.890,152f1b77a32508570e2745daf9ce7aec,69.890,40.190,Biweekly,40,8230aaeb57e52b80413424a7ac1b5f88,2020-01-03 00:00:00.000000,2020-01-22 00:00:00.000000,train,1728000000000000,2019-12-14 00:00:00.000000
3,3,0d4545df48da4bc3f6c15ea998174468,2019-12-30 00:00:00.000000,d39934ce111a864abf40391f3da9cdf5,9dcc271fc6d9f6492002d2ee14be9496,16:46:34,8230aaeb57e52b80413424a7ac1b5f88,25685af065808d88a7d8f16dbed8a87e,5,2.000,82.930,139.480,152f1b77a32508570e2745daf9ce7aec,69.740,41.465,Biweekly,40,8230aaeb57e52b80413424a7ac1b5f88,2020-01-03 00:00:00.000000,2020-01-22 00:00:00.000000,train,1728000000000000,2019-12-14 00:00:00.000000
4,4,60aa4d3f7bb66c05b568f1feb553a861,2019-12-30 00:00:00.000000,d39934ce111a864abf40391f3da9cdf5,0baf0815c297e9c619d359d1602e3e0b,18:24:54,9c8af54d8abde6b87cea2264ccfd9208,680a96609351230ed00e1d1090371cbc,16,1.000,125.190,99.990,152f1b77a32508570e2745daf9ce7aec,99.990,125.190,Biweekly,42,9c8af54d8abde6b87cea2264ccfd9208,2020-01-03 00:00:00.000000,2020-01-22 00:00:00.000000,train,1728000000000000,2019-12-14 00:00:00.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164647,164647,461b98b05cb38d8f6d4dc5dd73dff840,2020-01-01 00:00:00.000000,087c8abfaee44ebbf0c2871976a2ab18,3f8c7f0c64530ba22961303e75bb75c2,15:00:20,33306ccd877e73c388652739869d3dc7,bbc8891e052cddce4ea874493e46d74e,15,1.332,114.500,126.530,7e35e74e610188414ad24235dd787c78,94.992,85.961,Facades,F1,33306ccd877e73c388652739869d3dc7,2020-01-02 00:00:00.000000,2020-01-08 00:00:00.000000,train,604800000000000,2019-12-26 00:00:00.000000
164648,164648,461b98b05cb38d8f6d4dc5dd73dff840,2020-01-01 00:00:00.000000,087c8abfaee44ebbf0c2871976a2ab18,3f8c7f0c64530ba22961303e75bb75c2,15:00:20,33306ccd877e73c388652739869d3dc7,bbc8891e052cddce4ea874493e46d74e,15,1.332,114.500,126.530,7e35e74e610188414ad24235dd787c78,94.992,85.961,Facades,F252,33306ccd877e73c388652739869d3dc7,2020-01-02 00:00:00.000000,2020-01-08 00:00:00.000000,train,604800000000000,2019-12-26 00:00:00.000000
164649,164649,4e07c3d8698433c19a1ec1e82f06333b,2020-01-01 00:00:00.000000,087c8abfaee44ebbf0c2871976a2ab18,338317b221994c988cf1fc3fc2f4abea,20:52:10,6605a6d640755de8493c3c1c4a92db41,0,9,2.000,53.790,95.780,7e35e74e610188414ad24235dd787c78,47.890,26.895,Biweekly,38,6605a6d640755de8493c3c1c4a92db41,2020-01-03 00:00:00.000000,2020-01-22 00:00:00.000000,train,1728000000000000,2019-12-14 00:00:00.000000
164650,164650,5bc94ab094e54573f98c249bf0e3005c,2020-01-01 00:00:00.000000,087c8abfaee44ebbf0c2871976a2ab18,36871e0ccf7f7c047c8736f65389a47a,21:10:23,b03e2331082a68124c00baf9611a2dae,2c1148d66a537cd19a58a41e2f223225,2,1.000,30.110,45.440,7e35e74e610188414ad24235dd787c78,45.440,30.110,Biweekly,38,b03e2331082a68124c00baf96

In [29]:
checks_non_action.to_csv('checks_non_action.csv')

In [30]:
checks_with_action = checks_merged

In [32]:
checks_with_action = checks_with_action.merge(hierarchy, how='outer', on = ['sku'])

В полученных данных не будет данных для тестовой выборки, т.к. у нас нет данных за 4 квартал 2020, поэтому отдельно займёмся подготовкой признаков для тестовой выборки. 

In [37]:
checks_with_offers = checks_merged1.merge(offers, how='outer', on = ['sku'])

In [38]:
checks_with_offers = checks_with_offers[checks_with_offers['Offer_ID'].isna() != True]

In [39]:
checks_with_offers_test = checks_with_offers.query('train_test_group in ("test")')

In [41]:
checks_with_offers_test_pivot = checks_with_offers_test.pivot_table(index=['Offer_ID', 'sku', 'Promo_type'], values=['selling_price_per_unit', 'supplier_price_per_unit'] , aggfunc=['mean'])
checks_with_offers_test_pivot.columns = ['selling_price_per_unit', 'supplier_price_per_unit']
checks_with_offers_test_pivot = checks_with_offers_test_pivot.reset_index()
checks_with_offers_test_pivot

,Offer_ID,sku,Promo_type,selling_price_per_unit,supplier_price_per_unit
0,100,12e3ac7c9bf202d41ec92d747d92a234,Seasonal,287.310,238.005
1,100,228f88c28e3b259103f548e4f7788904,Seasonal,181.682,170.507
2,100,3e0fb22a7c69510b08869627f6214003,Seasonal,170.661,181.712
3,100,4d1581a093a0eb7751e1e0f62dcbe278,Seasonal,167.720,182.587
4,100,752e96b000091940545ce06692bf67c5,Seasonal,171.978,182.297
...,...,...,...,...,...
2055,F220,b13f4c65a7eaa1b23f45592b4c584552,Facades,218.563,132.879
2056,F220,b637b39fc09e0f72102968f4a501e164,Facades,297.476,220.153
2057,F220,ddff4e4054cdbdffafbc2963e5b6a997,Facades,293.083,211.446
2058,F220,f3d823fb9ac9a3f7a4e981f0bf20e40f,Facades,289.073,210.718


In [42]:
checks_with_offers_test_pivot.to_csv('checks_with_offers_test_pivot.csv')

Создадим сводные таблицы по трейн-выборке для создания датасета с признаками для обучения модели.

In [43]:
checks_with_action_pivot_price = checks_with_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type'], values=['selling_price_per_unit', 'supplier_price_per_unit'] , aggfunc=['mean'])
checks_with_action_pivot_price.columns = ['mean_selling_price_per_unit_action', 'mean_supplier_price_per_unit_action']
checks_with_action_pivot_price = checks_with_action_pivot_price.reset_index()
checks_with_action_pivot_price

,Offer_ID,sku,region_name,Promo_type,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.290,28.623
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,48.832,28.504
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.355,28.832
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,49.372,28.428
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.346,31.137
...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,115.438,110.175
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,71.952,48.791
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,81.594,38.961
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,29.943,22.317


In [44]:
checks_with_action_pivot_amount = checks_with_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type'], values=['num_sales', 'supplier_price', 'selling_price'] , aggfunc=['sum'])
checks_with_action_pivot_amount.columns = ['num_sales_action', 'selling_price_action', 'supplier_price_action']
checks_with_action_pivot_amount = checks_with_action_pivot_amount.reset_index()
checks_with_action_pivot_amount

,Offer_ID,sku,region_name,Promo_type,num_sales_action,selling_price_action,supplier_price_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,95.000,4684.180,2715.750
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,103.000,5036.900,2936.440
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,289.000,14265.200,8327.790
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,521.000,25722.260,14813.040
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,111.000,5477.810,3457.240
...,...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,3299.344,380506.710,363546.990
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,773.460,55035.570,37949.550
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,319.296,25920.530,12437.100
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,2511.510,75197.620,56061.950


In [45]:
checks_non_action_pivot_price = checks_non_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type'], values=['selling_price_per_unit', 'supplier_price_per_unit'] , aggfunc=['mean'])
checks_non_action_pivot_price.columns = ['mean_selling_price_per_unit_without_action', 'mean_supplier_price_per_unit_without_action']
checks_non_action_pivot_price = checks_non_action_pivot_price.reset_index()
checks_non_action_pivot_price

,Offer_ID,sku,region_name,Promo_type,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,77.996,29.990
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,74.865,28.818
2,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,82.490,30.930
3,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,83.990,30.930
4,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,83.990,30.983
...,...,...,...,...,...,...
12704,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,141.456,105.032
12705,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,74.711,47.297
12706,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,83.753,37.983
12707,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,38.316,26.490


In [46]:
checks_non_action_pivot_amount = checks_non_action.pivot_table(index=['Offer_ID', 'sku', 'region_name', 'Promo_type'], values=['num_sales', 'supplier_price', 'selling_price'] , aggfunc=['sum'])
checks_non_action_pivot_amount.columns = ['num_sales_without_action', 'selling_price_without_action', 'supplier_price_without_action']
checks_non_action_pivot_amount = checks_non_action_pivot_amount.reset_index()
checks_non_action_pivot_amount

,Offer_ID,sku,region_name,Promo_type,num_sales_without_action,selling_price_without_action,supplier_price_without_action
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,20.000,1562.700,598.590
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,34.000,2537.990,986.540
2,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,2.000,164.980,61.860
3,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,1.000,83.990,30.930
4,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,5.000,419.950,154.860
...,...,...,...,...,...,...,...
12704,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,506.986,71652.860,53380.760
12705,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,503.724,37428.130,23808.970
12706,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,321.260,26918.890,12243.790
12707,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,685.386,26440.340,18299.620


In [47]:
checks_non_action_merged = checks_non_action_pivot_price.merge(checks_non_action_pivot_amount, how='outer', on = ['Offer_ID', 'sku', 'region_name'])
checks_non_action_merged

,Offer_ID,sku,region_name,Promo_type_x,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540
2,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860
3,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,83.990,30.930,Biweekly,1.000,83.990,30.930
4,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,83.990,30.983,Biweekly,5.000,419.950,154.860
...,...,...,...,...,...,...,...,...,...,...
12704,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,141.456,105.032,Facades,506.986,71652.860,53380.760
12705,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,74.711,47.297,Facades,503.724,37428.130,23808.970
12706,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,83.753,37.983,Facades,321.260,26918.890,12243.790
12707,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,38.316,26.490,Facades,685.386,26440.340,18299.620


In [48]:
checks_with_action_merged = checks_with_action_pivot_price.merge(checks_with_action_pivot_amount, how='outer', on = ['Offer_ID', 'sku', 'region_name'])
checks_with_action_merged

,Offer_ID,sku,region_name,Promo_type_x,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y,num_sales_action,selling_price_action,supplier_price_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.290,28.623,Biweekly,95.000,4684.180,2715.750
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,48.832,28.504,Biweekly,103.000,5036.900,2936.440
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.355,28.832,Biweekly,289.000,14265.200,8327.790
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,49.372,28.428,Biweekly,521.000,25722.260,14813.040
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.346,31.137,Biweekly,111.000,5477.810,3457.240
...,...,...,...,...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,115.438,110.175,Facades,3299.344,380506.710,363546.990
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,71.952,48.791,Facades,773.460,55035.570,37949.550
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,81.594,38.961,Facades,319.296,25920.530,12437.100
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,29.943,22.317,Facades,2511.510,75197.620,56061.950


In [49]:
checks_offers_merged = checks_with_action_merged.merge(checks_non_action_merged, how='outer', on = ['Offer_ID', 'sku', 'region_name'])
checks_offers_merged

,Offer_ID,sku,region_name,Promo_type_x_x,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.290,28.623,Biweekly,95.000,4684.180,2715.750,NaN,nan,nan,NaN,nan,nan,nan
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,48.832,28.504,Biweekly,103.000,5036.900,2936.440,NaN,nan,nan,NaN,nan,nan,nan
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14769,F5,edbe917ef00874205ad893c3931ea9e0,7e35e74e610188414ad24235dd787c78,NaN,nan,nan,NaN,nan,nan,nan,Facades,101.990,95.600,Facades,1.000,101.990,95.600
14770,F50,f3f807ff0fe767b94827bc9d8367d324,152f1b77a32508570e2745daf9ce7aec,NaN,nan,nan,NaN,nan,nan,nan,Facades,188.990,95.240,Facades,1.000,188.990,95.240
14771,F50,fe1d8a29a7ed76cd79f0a520ed18e0e6,152f1b77a32508570e2745daf9ce7aec,NaN,nan,nan,NaN,nan,nan,nan,Facades,188.410,101.300,Facades,1.000,188.410,101.300
14772,F53,6b046ae5ddceb4ca2dca623fbc9011cf,7e35e74e610188414ad24235dd787c78,NaN,nan,nan,NaN,nan,nan,nan,Facades,74.890,57.280,Facades,2.000,149.780,114.560


In [50]:
checks_offers_merged_without_null = checks_offers_merged.dropna()
checks_offers_merged_without_null.head(10)

,Offer_ID,sku,region_name,Promo_type_x_x,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860
5,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,49.387,31.173,Biweekly,174.000,8592.680,5419.670,Biweekly,83.990,30.930,Biweekly,1.000,83.990,30.930
6,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.369,31.012,Biweekly,144.000,7108.950,4464.550,Biweekly,83.990,30.983,Biweekly,5.000,419.950,154.860
7,10,66dd48b798614b7d193a83777c8a0fdc,7e35e74e610188414ad24235dd787c78,Biweekly,49.337,31.173,Biweekly,193.000,9519.540,6013.880,Biweekly,83.990,31.580,Biweekly,2.000,167.980,63.160
8,10,953fce3d637849721d74508251737be2,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.303,31.570,Biweekly,195.000,9617.940,6146.010,Biweekly,71.443,34.082,Biweekly,7.000,511.050,237.360
9,10,953fce3d637849721d74508251737be2,7e35e74e610188414ad24235dd787c78,Biweekly,49.549,31.473,Biweekly,229.000,11367.570,7200.930,Biweekly,79.690,31.693,Biweekly,5.000,398.250,158.310
10,10,c4b0a5d6be61feef47a33f8230beec38,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.324,29.537,Biweekly,161.000,7941.000,4746.400,Biweekly,67.530,32.766,Biweekly,7.000,486.550,221.030
11,10,c4b0a5d6be61feef47a33f8230beec38,7e35e74e610188414ad24235dd787c78,Biweekly,49.050,29.133,Biweekly,250.000,12284.760,7278.250,Biweekly,81.672,29.196,Biweekly,12.000,972.400,350.670


In [51]:
checks_offers_merged_without_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12509 entries, 2 to 14573
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Offer_ID                                     12509 non-null  object 
 1   sku                                          12509 non-null  object 
 2   region_name                                  12509 non-null  object 
 3   Promo_type_x_x                               12509 non-null  object 
 4   mean_selling_price_per_unit_action           12509 non-null  float64
 5   mean_supplier_price_per_unit_action          12509 non-null  float64
 6   Promo_type_y_x                               12509 non-null  object 
 7   num_sales_action                             12509 non-null  float64
 8   selling_price_action                         12509 non-null  float64
 9   supplier_price_action                        12509 non-null  float64
 10

In [52]:
checks_offers = checks_offers_merged.merge(offers, how='outer', on = ['Offer_ID', 'sku'])
checks_offers

,Offer_ID,sku,region_name,Promo_type_x_x,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action,Promo_type,start_date,end_date,train_test_group,duration,pre_start_date
0,10,172239f5f436f6b6661fdaa10fb47e9f,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.290,28.623,Biweekly,95.000,4684.180,2715.750,NaN,nan,nan,NaN,nan,nan,nan,Biweekly,2020-04-02,2020-04-15,train,14 days,2020-03-19
1,10,172239f5f436f6b6661fdaa10fb47e9f,7e35e74e610188414ad24235dd787c78,Biweekly,48.832,28.504,Biweekly,103.000,5036.900,2936.440,NaN,nan,nan,NaN,nan,nan,nan,Biweekly,2020-04-02,2020-04-15,train,14 days,2020-03-19
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590,Biweekly,2020-04-02,2020-04-15,train,14 days,2020-03-19
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540,Biweekly,2020-04-02,2020-04-15,train,14 days,2020-03-19
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860,Biweekly,2020-04-02,2020-04-15,train,14 days,2020-03-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16962,646,9947c0a36935be2fb8dadcf945f46bdb,NaN,NaN,nan,nan,NaN,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,nan,Seasonal,2019-10-08,2019-10-21,train,14 days,2019-09-24
16963,646,651a1cd90cd946ba5d59bdfc18e5f7d6,NaN,NaN,nan,nan,NaN,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,nan,Seasonal,2019-10-08,2019-10-21,train,14 days,2019-09-24
16964,658,4ab74754a9f4cec1f07ab128088d79d3,NaN,NaN,nan,nan,NaN,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,nan,Biweekly,2019-10-17,2019-10-30,train,14 days,2019-10-03
16965,679,2efbf8fe13ce1151ac975f1c4085f023,NaN,NaN,nan,nan,NaN,nan,nan,nan,NaN,nan,nan,NaN,nan,nan,nan,Biweekly,2019-10-31,2019-11-13,train,14 days,2019-10-17


In [53]:
checks_offers_merged_without_null['uplift'] = checks_offers_merged_without_null['selling_price_action'] / checks_offers_merged_without_null['selling_price_without_action'] - 1
checks_offers_merged_without_null

,Offer_ID,sku,region_name,Promo_type_x_x,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action,uplift
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590,8.129
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540,9.135
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860,32.203
5,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,49.387,31.173,Biweekly,174.000,8592.680,5419.670,Biweekly,83.990,30.930,Biweekly,1.000,83.990,30.930,101.306
6,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.369,31.012,Biweekly,144.000,7108.950,4464.550,Biweekly,83.990,30.983,Biweekly,5.000,419.950,154.860,15.928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14569,F98,ef7bf412117548cb80b3f3b33a12977d,7e35e74e610188414ad24235dd787c78,Facades,115.438,110.175,Facades,3299.344,380506.710,363546.990,Facades,141.456,105.032,Facades,506.986,71652.860,53380.760,4.310
14570,F99,5e3039a269ff005f2c76cea16c0cf783,152f1b77a32508570e2745daf9ce7aec,Facades,71.952,48.791,Facades,773.460,55035.570,37949.550,Facades,74.711,47.297,Facades,503.724,37428.130,23808.970,0.470
14571,F99,5e3039a269ff005f2c76cea16c0cf783,7e35e74e610188414ad24235dd787c78,Facades,81.594,38.961,Facades,319.296,25920.530,12437.100,Facades,83.753,37.983,Facades,321.260,26918.890,12243.790,-0.037
14572,F99,6be0907d36fbbf5f7378f32c46f37943,152f1b77a32508570e2745daf9ce7aec,Facades,29.943,22.317,Facades,2511.510,75197.620,56061.950,Facades,38.316,26.490,Facades,685.386,26440.340,18299.620,1.844


In [56]:
checks_offers_merged_without_null_offerid = checks_offers_merged_without_null.pivot_table(index=['Offer_ID'], values=['selling_price_action', 'selling_price_without_action'] , aggfunc=['sum'])
checks_offers_merged_without_null_offerid.columns = ['selling_price_action', 'selling_price_without_action']
checks_offers_merged_without_null_offerid = checks_offers_merged_without_null_offerid.reset_index()
checks_offers_merged_without_null_offerid

,Offer_ID,selling_price_action,selling_price_without_action
0,10,138180.700,8730.100
1,101,44033.640,15945.010
2,102,184459.580,113132.790
3,103,58245.910,16617.230
4,104,125074.530,167879.360
...,...,...,...
735,F95,179631.800,56673.800
736,F96,97151.720,69313.070
737,F97,121617.680,37121.460
738,F98,885846.520,341628.680


In [57]:
checks_offers_merged_without_null_offerid['uplift'] = checks_offers_merged_without_null_offerid['selling_price_action'] / checks_offers_merged_without_null_offerid['selling_price_without_action'] - 1
checks_offers_merged_without_null_offerid

,Offer_ID,selling_price_action,selling_price_without_action,uplift
0,10,138180.700,8730.100,14.828
1,101,44033.640,15945.010,1.762
2,102,184459.580,113132.790,0.630
3,103,58245.910,16617.230,2.505
4,104,125074.530,167879.360,-0.255
...,...,...,...,...
735,F95,179631.800,56673.800,2.170
736,F96,97151.720,69313.070,0.402
737,F97,121617.680,37121.460,2.276
738,F98,885846.520,341628.680,1.593


In [58]:
checks_offers_uplift = checks_offers_merged_without_null_offerid.merge(uplift, how='outer', on = ['Offer_ID'])
checks_offers_uplift

,Offer_ID,selling_price_action,selling_price_without_action,uplift,UpLift
0,10,138180.700,8730.100,14.828,15.942
1,101,44033.640,15945.010,1.762,1.762
2,102,184459.580,113132.790,0.630,0.630
3,103,58245.910,16617.230,2.505,2.505
4,104,125074.530,167879.360,-0.255,-0.255
...,...,...,...,...,...
735,F95,179631.800,56673.800,2.170,2.170
736,F96,97151.720,69313.070,0.402,0.402
737,F97,121617.680,37121.460,2.276,2.276
738,F98,885846.520,341628.680,1.593,1.593


In [ ]:
checks_with_offers_test_pivot = pd.read_csv('checks_with_offers_test_pivot.csv')

In [59]:
checks_with_offers_test_pivot['margin'] = checks_with_offers_test_pivot['selling_price_per_unit'] / checks_with_offers_test_pivot['supplier_price_per_unit'] - 1

In [60]:
checks_with_offers_test_pivot_merged = checks_with_offers_test_pivot.merge(hierarchy, how='inner', on = ['sku'])
checks_with_offers_test_pivot_merged

,Offer_ID,sku,Promo_type,selling_price_per_unit,supplier_price_per_unit,margin,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,100,12e3ac7c9bf202d41ec92d747d92a234,Seasonal,287.310,238.005,0.207,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
1,100,228f88c28e3b259103f548e4f7788904,Seasonal,181.682,170.507,0.066,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,1400ffe327b1b15130fe85535eee58ba
2,100,3e0fb22a7c69510b08869627f6214003,Seasonal,170.661,181.712,-0.061,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
3,100,4d1581a093a0eb7751e1e0f62dcbe278,Seasonal,167.720,182.587,-0.081,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
4,100,752e96b000091940545ce06692bf67c5,Seasonal,171.978,182.297,-0.057,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
...,...,...,...,...,...,...,...,...,...,...
2055,F220,b13f4c65a7eaa1b23f45592b4c584552,Facades,218.563,132.879,0.645,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2056,F220,b637b39fc09e0f72102968f4a501e164,Facades,297.476,220.153,0.351,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2057,F220,ddff4e4054cdbdffafbc2963e5b6a997,Facades,293.083,211.446,0.386,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2058,F220,f3d823fb9ac9a3f7a4e981f0bf20e40f,Facades,289.073,210.718,0.372,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d


In [61]:
checks_with_offers_test_pivot_merged_offerid = checks_with_offers_test_pivot_merged.pivot_table(index=['Offer_ID', 'Promo_type'], values=['selling_price_per_unit', 'supplier_price_per_unit', 'margin'] , aggfunc=['mean'])
checks_with_offers_test_pivot_merged_offerid.columns = ['margin', 'selling_price_per_unit', 'supplier_price_per_unit', ]
checks_with_offers_test_pivot_merged_offerid = checks_with_offers_test_pivot_merged_offerid.reset_index()
checks_with_offers_test_pivot_merged_offerid

,Offer_ID,Promo_type,margin,selling_price_per_unit,supplier_price_per_unit
0,100,Seasonal,0.086,222.123,203.559
1,172,Biweekly,0.176,197.775,168.234
2,173,Biweekly,0.752,220.045,128.284
3,174,Biweekly,0.683,100.695,59.943
4,175,Biweekly,0.028,144.794,141.161
...,...,...,...,...,...
144,F216,Facades,0.226,43.723,35.875
145,F217,Facades,0.676,49.172,29.509
146,F218,Facades,0.165,103.323,88.783
147,F219,Facades,0.488,170.549,114.509


In [68]:
test = checks_with_offers_test_pivot_merged
test

,Offer_ID,sku,Promo_type,selling_price_per_unit,supplier_price_per_unit,margin,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,100,12e3ac7c9bf202d41ec92d747d92a234,Seasonal,287.310,238.005,0.207,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
1,100,228f88c28e3b259103f548e4f7788904,Seasonal,181.682,170.507,0.066,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,1400ffe327b1b15130fe85535eee58ba
2,100,3e0fb22a7c69510b08869627f6214003,Seasonal,170.661,181.712,-0.061,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
3,100,4d1581a093a0eb7751e1e0f62dcbe278,Seasonal,167.720,182.587,-0.081,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
4,100,752e96b000091940545ce06692bf67c5,Seasonal,171.978,182.297,-0.057,44a7b825383402ed937a15fb8a5bb12a,d67d8ab4f4c10bf22aa353e27879133c,1be883eec3231f9fe43c35bd1b4b3bb5,879848e74e5c4ad8d310bd70bbdce0a5
...,...,...,...,...,...,...,...,...,...,...
2055,F220,b13f4c65a7eaa1b23f45592b4c584552,Facades,218.563,132.879,0.645,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2056,F220,b637b39fc09e0f72102968f4a501e164,Facades,297.476,220.153,0.351,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2057,F220,ddff4e4054cdbdffafbc2963e5b6a997,Facades,293.083,211.446,0.386,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d
2058,F220,f3d823fb9ac9a3f7a4e981f0bf20e40f,Facades,289.073,210.718,0.372,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,9ef2ed4b7fd2c810847ffa5fa85bce38,e36062571db5523ef17e546b7dd14e9d


In [77]:
checks_offers_merged_without_null.head()

,Offer_ID,sku,region_name,Promo_type_x_x,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action,uplift,margin
2,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590,8.129,1.601
3,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540,9.135,1.598
4,10,376f806100fa4c1eb6c3878ead12631d,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.346,31.137,Biweekly,111.000,5477.810,3457.240,Biweekly,82.490,30.930,Biweekly,2.000,164.980,61.860,32.203,1.667
5,10,376f806100fa4c1eb6c3878ead12631d,7e35e74e610188414ad24235dd787c78,Biweekly,49.387,31.173,Biweekly,174.000,8592.680,5419.670,Biweekly,83.990,30.930,Biweekly,1.000,83.990,30.930,101.306,1.715
6,10,66dd48b798614b7d193a83777c8a0fdc,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.369,31.012,Biweekly,144.000,7108.950,4464.550,Biweekly,83.990,30.983,Biweekly,5.000,419.950,154.860,15.928,1.711


In [62]:
checks_offers_merged_without_null['margin'] = checks_offers_merged_without_null['mean_selling_price_per_unit_without_action'] / checks_offers_merged_without_null['mean_supplier_price_per_unit_without_action'] - 1

In [63]:
checks_offers_merged_without_null_h = checks_offers_merged_without_null.merge(hierarchy, how='inner', on = ['sku'])
checks_offers_merged_without_null_h

,Offer_ID,sku,region_name,Promo_type_x_x,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action,uplift,margin,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590,8.129,1.601,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540,9.135,1.598,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
2,F137,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Facades,49.380,30.773,Facades,113.000,5579.960,3459.870,Facades,74.404,29.826,Facades,11.000,794.580,328.720,6.023,1.495,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
3,F137,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Facades,49.368,29.893,Facades,126.000,6220.320,3755.490,Facades,83.953,29.529,Facades,5.000,416.240,148.760,13.944,1.843,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
4,F287,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Facades,44.465,30.653,Facades,135.000,5994.560,4136.210,Facades,73.687,30.839,Facades,7.000,583.410,216.760,9.275,1.389,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,F97,9794c588b279cd37153e81d9109acf70,7e35e74e610188414ad24235dd787c78,Facades,44.027,31.706,Facades,25.000,1100.870,783.020,Facades,61.590,27.140,Facades,4.000,246.360,108.560,3.469,1.269,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12505,F97,aa743eaaf303d1ded42c543797089f35,152f1b77a32508570e2745daf9ce7aec,Facades,44.082,28.774,Facades,74.000,3262.260,2122.640,Facades,70.291,33.871,Facades,23.000,1656.810,755.510,0.969,1.075,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12506,F97,aa743eaaf303d1ded42c543797089f35,7e35e74e610188414ad24235dd787c78,Facades,44.080,27.922,Facades,48.000,2115.650,1338.390,Facades,75.270,31.369,Facades,8.000,602.160,250.950,2.513,1.400,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12507,F97,d9b38fad6052a1c1045b18b67f34978b,152f1b77a32508570e2745daf9ce7aec,Facades,273.589,163.764,Facades,21.000,5745.370,3439.040,Facades,383.010,180.750,Facades,3.000,1149.030,542.250,4.000,1.119,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,7599d4e0d9b5eae4fecdc440358604b1


In [64]:
checks_offers_merged_without_null_h = checks_offers_merged_without_null.merge(hierarchy, how='inner', on = ['sku'])
checks_offers_merged_without_null_h

,Offer_ID,sku,region_name,Promo_type_x_x,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action,uplift,margin,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590,8.129,1.601,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540,9.135,1.598,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
2,F137,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Facades,49.380,30.773,Facades,113.000,5579.960,3459.870,Facades,74.404,29.826,Facades,11.000,794.580,328.720,6.023,1.495,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
3,F137,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Facades,49.368,29.893,Facades,126.000,6220.320,3755.490,Facades,83.953,29.529,Facades,5.000,416.240,148.760,13.944,1.843,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
4,F287,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Facades,44.465,30.653,Facades,135.000,5994.560,4136.210,Facades,73.687,30.839,Facades,7.000,583.410,216.760,9.275,1.389,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,F97,9794c588b279cd37153e81d9109acf70,7e35e74e610188414ad24235dd787c78,Facades,44.027,31.706,Facades,25.000,1100.870,783.020,Facades,61.590,27.140,Facades,4.000,246.360,108.560,3.469,1.269,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12505,F97,aa743eaaf303d1ded42c543797089f35,152f1b77a32508570e2745daf9ce7aec,Facades,44.082,28.774,Facades,74.000,3262.260,2122.640,Facades,70.291,33.871,Facades,23.000,1656.810,755.510,0.969,1.075,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12506,F97,aa743eaaf303d1ded42c543797089f35,7e35e74e610188414ad24235dd787c78,Facades,44.080,27.922,Facades,48.000,2115.650,1338.390,Facades,75.270,31.369,Facades,8.000,602.160,250.950,2.513,1.400,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12507,F97,d9b38fad6052a1c1045b18b67f34978b,152f1b77a32508570e2745daf9ce7aec,Facades,273.589,163.764,Facades,21.000,5745.370,3439.040,Facades,383.010,180.750,Facades,3.000,1149.030,542.250,4.000,1.119,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,7599d4e0d9b5eae4fecdc440358604b1


In [66]:
train = checks_offers_merged_without_null_h
train

,Offer_ID,sku,region_name,Promo_type_x_x,mean_selling_price_per_unit_action,mean_supplier_price_per_unit_action,Promo_type_y_x,num_sales_action,selling_price_action,supplier_price_action,Promo_type_x_y,mean_selling_price_per_unit_without_action,mean_supplier_price_per_unit_without_action,Promo_type_y_y,num_sales_without_action,selling_price_without_action,supplier_price_without_action,uplift,margin,hierarchy_level1,hierarchy_level2,hierarchy_level3,hierarchy_level4
0,10,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Biweekly,49.355,28.832,Biweekly,289.000,14265.200,8327.790,Biweekly,77.996,29.990,Biweekly,20.000,1562.700,598.590,8.129,1.601,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
1,10,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Biweekly,49.372,28.428,Biweekly,521.000,25722.260,14813.040,Biweekly,74.865,28.818,Biweekly,34.000,2537.990,986.540,9.135,1.598,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
2,F137,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Facades,49.380,30.773,Facades,113.000,5579.960,3459.870,Facades,74.404,29.826,Facades,11.000,794.580,328.720,6.023,1.495,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
3,F137,1f24b678deda0ec059c12a327eba8ca9,7e35e74e610188414ad24235dd787c78,Facades,49.368,29.893,Facades,126.000,6220.320,3755.490,Facades,83.953,29.529,Facades,5.000,416.240,148.760,13.944,1.843,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
4,F287,1f24b678deda0ec059c12a327eba8ca9,152f1b77a32508570e2745daf9ce7aec,Facades,44.465,30.653,Facades,135.000,5994.560,4136.210,Facades,73.687,30.839,Facades,7.000,583.410,216.760,9.275,1.389,cc9284597026dcc021735c0970ca0c93,6364d3f0f495b6ab9dcf8d3b5c6e0b01,7486cef2522ee03547cfb970a404a874,3f6ab5dd77f73b11f122ac7c0fe2de76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,F97,9794c588b279cd37153e81d9109acf70,7e35e74e610188414ad24235dd787c78,Facades,44.027,31.706,Facades,25.000,1100.870,783.020,Facades,61.590,27.140,Facades,4.000,246.360,108.560,3.469,1.269,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12505,F97,aa743eaaf303d1ded42c543797089f35,152f1b77a32508570e2745daf9ce7aec,Facades,44.082,28.774,Facades,74.000,3262.260,2122.640,Facades,70.291,33.871,Facades,23.000,1656.810,755.510,0.969,1.075,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12506,F97,aa743eaaf303d1ded42c543797089f35,7e35e74e610188414ad24235dd787c78,Facades,44.080,27.922,Facades,48.000,2115.650,1338.390,Facades,75.270,31.369,Facades,8.000,602.160,250.950,2.513,1.400,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,c9433f08a67b8c18ff388484beee4efa
12507,F97,d9b38fad6052a1c1045b18b67f34978b,152f1b77a32508570e2745daf9ce7aec,Facades,273.589,163.764,Facades,21.000,5745.370,3439.040,Facades,383.010,180.750,Facades,3.000,1149.030,542.250,4.000,1.119,cc9284597026dcc021735c0970ca0c93,37693cfc748049e45d87b8c7d8b9aacd,db9eeb7e678863649bce209842e0d164,7599d4e0d9b5eae4fecdc440358604b1


## Моделирование

## Модель CatBoost

In [69]:
features_train = train[['Offer_ID', 'sku', 'Promo_type_x_x', 'mean_selling_price_per_unit_without_action', 'mean_supplier_price_per_unit_without_action', 'margin', 'hierarchy_level1', 'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']]
target_train = train['uplift']
features_test = test[['Offer_ID', 'sku', 'Promo_type', 'selling_price_per_unit', 'supplier_price_per_unit', 'margin', 'hierarchy_level1', 'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']]

In [70]:
features_train.columns = ['Offer_ID', 'sku', 'Promo_type', 'selling_price_per_unit', 'supplier_price_per_unit', 'margin', 'hierarchy_level1', 'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']

In [71]:
cat_feature = ['Offer_ID', 'sku', 'Promo_type', 'hierarchy_level1', 'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']

In [72]:
model_cat = CatBoostRegressor(loss_function='MAE', random_seed=12345, iterations=150)
model_cat.fit(features_train, target_train, cat_features=cat_feature, verbose=10)

0:	learn: 4.1699663	total: 58.8ms	remaining: 8.77s
10:	learn: 3.8562565	total: 92.9ms	remaining: 1.17s
20:	learn: 3.6660869	total: 125ms	remaining: 768ms
30:	learn: 3.5379162	total: 157ms	remaining: 603ms
40:	learn: 3.4550010	total: 189ms	remaining: 503ms
50:	learn: 3.3953390	total: 221ms	remaining: 430ms
60:	learn: 3.3509172	total: 254ms	remaining: 370ms
70:	learn: 3.3202310	total: 286ms	remaining: 319ms
80:	learn: 3.2968617	total: 318ms	remaining: 271ms
90:	learn: 3.2795497	total: 350ms	remaining: 227ms
100:	learn: 3.2622187	total: 382ms	remaining: 185ms
110:	learn: 3.2475770	total: 414ms	remaining: 145ms
120:	learn: 3.2338049	total: 445ms	remaining: 107ms
130:	learn: 3.2186017	total: 478ms	remaining: 69.3ms
140:	learn: 3.2084171	total: 510ms	remaining: 32.6ms
149:	learn: 3.1966203	total: 539ms	remaining: 0us


In [73]:
predictions_cat_train = model_cat.predict(features_train)
predictions_cat_test = model_cat.predict(features_test)

In [74]:
result_cat = mean_absolute_error(target_train, predictions_cat_train)
print("MAE для catboost:", result_cat)

MAE для catboost: 3.0309072520072267


In [75]:
test['UpLift'] = predictions_cat_test

In [78]:
test_final1 = test.pivot_table(index=['Offer_ID'], values=['UpLift'] , aggfunc=['mean'])
test_final1.columns = ['UpLift']
test_final1 = test_final1.reset_index()
test_final1

,Offer_ID,UpLift
0,100,2.277
1,172,3.311
2,173,9.052
3,174,3.433
4,175,1.969
...,...,...
144,F216,1.116
145,F217,1.515
146,F218,0.781
147,F219,3.724


In [80]:
test_final1.to_csv('test_final1.csv', index = None)

## Модель LightGMB

In [81]:
features_train['Offer_ID'] = features_train['Offer_ID'].astype('category')
features_test['Offer_ID'] = features_test['Offer_ID'].astype('category')
features_train['sku'] = features_train['sku'].astype('category')
features_test['sku'] = features_test['sku'].astype('category')
features_train['Promo_type'] = features_train['Promo_type'].astype('category')
features_test['Promo_type'] = features_test['Promo_type'].astype('category')
features_train['hierarchy_level1'] = features_train['hierarchy_level1'].astype('category')
features_test['hierarchy_level1'] = features_test['hierarchy_level1'].astype('category')
features_train['hierarchy_level2'] = features_train['hierarchy_level2'].astype('category')
features_test['hierarchy_level2'] = features_test['hierarchy_level2'].astype('category')
features_train['hierarchy_level3'] = features_train['hierarchy_level3'].astype('category')
features_test['hierarchy_level3'] = features_test['hierarchy_level3'].astype('category')
features_train['hierarchy_level4'] = features_train['hierarchy_level4'].astype('category')
features_test['hierarchy_level4'] = features_test['hierarchy_level4'].astype('category')

In [82]:
cat_feature = ['Offer_ID', 'sku', 'Promo_type', 'hierarchy_level1', 'hierarchy_level2', 'hierarchy_level3', 'hierarchy_level4']
model_light = LGBMRegressor(num_leaves = 31, max_depth = -1, random_state=12345, n_estimators=100, categorical_feature = cat_feature)
model_light.fit(features_train, target_train, verbose=10)

LGBMRegressor(boosting_type='gbdt',
              categorical_feature=['Offer_ID', 'sku', 'Promo_type',
                                   'hierarchy_level1', 'hierarchy_level2',
                                   'hierarchy_level3', 'hierarchy_level4'],
              class_weight=None, colsample_bytree=1.0, importance_type='split',
              learning_rate=0.1, max_depth=-1, min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_leaves=31, objective=None, random_state=12345,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [83]:
predictions_light_train = model_light.predict(features_train)

In [84]:
result_light = mean_absolute_error(target_train, predictions_light_train)
print("MAE для lightgmb:", result_light)

MAE для lightgmb: 2.614464294496264


In [85]:
predictions_light_test = model_light.predict(features_test)

In [86]:
test['UpLift'] = predictions_light_test

In [87]:
test_final2 = test.pivot_table(index=['Offer_ID'], values=['UpLift'] , aggfunc=['mean'])
test_final2.columns = ['UpLift']
test_final2 = test_final2.reset_index()
test_final2

,Offer_ID,UpLift
0,100,4.605
1,172,2.711
2,173,11.964
3,174,5.876
4,175,2.778
...,...,...
144,F216,2.266
145,F217,1.367
146,F218,1.245
147,F219,8.222


In [88]:
test_final2.to_csv('test_final2.csv', index = None)

## Модель CatBoost c одним фактором категориальной иерархии

In [89]:
features_train_с = train[['Offer_ID', 'sku', 'Promo_type_x_x', 'mean_selling_price_per_unit_without_action', 'mean_supplier_price_per_unit_without_action', 'margin', 'hierarchy_level4']]
target_train_c = train['uplift']
features_test_c = test[['Offer_ID', 'sku', 'Promo_type', 'selling_price_per_unit', 'supplier_price_per_unit', 'margin', 'hierarchy_level4']]

In [90]:
features_train_с.columns = ['Offer_ID', 'sku', 'Promo_type', 'selling_price_per_unit', 'supplier_price_per_unit', 'margin', 'hierarchy_level4']

In [91]:
cat_feature_c = ['Offer_ID', 'sku', 'Promo_type', 'hierarchy_level4']

In [93]:
model_cat_c = CatBoostRegressor(loss_function='MAE', random_seed=12345, iterations=150)
model_cat_c.fit(features_train_с, target_train_c, cat_features=cat_feature_c, verbose=10)

0:	learn: 4.1727178	total: 4.25ms	remaining: 633ms
10:	learn: 3.8632714	total: 35.7ms	remaining: 452ms
20:	learn: 3.6656014	total: 65.6ms	remaining: 403ms
30:	learn: 3.5366267	total: 95.2ms	remaining: 365ms
40:	learn: 3.4507296	total: 125ms	remaining: 331ms
50:	learn: 3.3888415	total: 157ms	remaining: 305ms
60:	learn: 3.3537376	total: 186ms	remaining: 271ms
70:	learn: 3.3245004	total: 217ms	remaining: 241ms
80:	learn: 3.3030765	total: 249ms	remaining: 212ms
90:	learn: 3.2870115	total: 279ms	remaining: 181ms
100:	learn: 3.2716977	total: 310ms	remaining: 150ms
110:	learn: 3.2561637	total: 339ms	remaining: 119ms
120:	learn: 3.2432779	total: 370ms	remaining: 88.7ms
130:	learn: 3.2318063	total: 401ms	remaining: 58.2ms
140:	learn: 3.2218250	total: 432ms	remaining: 27.6ms
149:	learn: 3.2117294	total: 460ms	remaining: 0us


In [94]:
predictions_cat_train_c = model_cat_c.predict(features_train_с)
predictions_cat_test_c = model_cat_c.predict(features_test_c)

In [95]:
result_cat_c = mean_absolute_error(target_train_c, predictions_cat_train_c)
print("MAE для catboost:", result_cat_c)

MAE для catboost: 3.0253986347531865


In [96]:
test['UpLift'] = predictions_cat_test_c

In [97]:
test_final3 = test.pivot_table(index=['Offer_ID'], values=['UpLift'] , aggfunc=['mean'])
test_final3.columns = ['UpLift']
test_final3 = test_final3.reset_index()
test_final3

,Offer_ID,UpLift
0,100,2.426
1,172,3.639
2,173,8.253
3,174,4.074
4,175,2.389
...,...,...
144,F216,0.831
145,F217,1.596
146,F218,0.901
147,F219,4.471


In [98]:
test_final3.to_csv('test_final3.csv', index = None)